Find the leading scorer by zone on the court. Plot the zones and players' names.

Plot the shooting charts of teams and/or players

In [6]:
import sys
import pandas as pd
from matplotlib import pyplot as plt
from euroleague_api.shot_data import ShotData

sys.path.append("../utils/")
from shot_chart_plots import plot_scatter, plot_leading_scorers_by_zone, fg_perc_hex_heatmap
from get_new_zones import hexagon_zones, corner_three_zones
# %matplotlib

plt.close("all")

In [7]:
def get_leading_scorer_by_zone(df, zone_col="ZONE"):
    """Calculate the leading scorers by zone

    Args:
        df (pd.DataFrame): The dataframe of the shooting data
        zone_col (str): The column name of the zone data.

    Returns:
        pd.DataFrame: A dataframe with the leading scorers by zone and their points
            scored in that zone
    """
    top_players_by_zone_df = df.groupby([zone_col, "PLAYER"])["POINTS"].sum().reset_index()
    pidx = top_players_by_zone_df.groupby(zone_col)['POINTS'].idxmax()
    top_players = top_players_by_zone_df.loc[pidx]
    # top_players.rename(columns={zone_col: "ZONE"}, inplace=True)
    return top_players


def reshape_actions_player_by_zone(df, zone_col):
    """Reshape the shooting dataframe and actions into a single FG statistic.

    Args:
        df (pd.DataFrame): The dataframe of the shooting data
        zone_col (str): The column name of the zone data.

    Returns:
        pd.DataFrame: A dataframe with zone, player name and their FGT and FG%
            statistics.
    """
    df["ID_ACTION_FG"] = df["ID_ACTION"].str.strip("2").str.strip("3")
    pvt_df = df.pivot_table(
        index=[zone_col, "PLAYER"],
        columns="ID_ACTION_FG",
        values="Season",
        aggfunc="count"
    ).fillna(0)
    pvt_df["FGT"] = pvt_df["FGA"] + pvt_df["FGM"]
    pvt_df["FG%"] = pvt_df["FGM"] / pvt_df["FGT"]
    pvt_df.reset_index(inplace=True)
    return pvt_df


def get_leading_attempts_by_zone(df, zone_col="ZONE"):
    """Calculate the leading volume shooters (total FGs) by zone

    Args:
        df (pd.DataFrame): The dataframe of the shooting data
        zone_col (str): The column name of the zone data.

    Returns:
        pd.DataFrame: A dataframe with the leading player of total attempted FG
            by zone
    """
    pvt_df = reshape_actions_player_by_zone(df, zone_col)

    pidx = pvt_df.groupby(zone_col)["FGT"].idxmax()
    top_players = pvt_df.loc[pidx]
    return top_players[[zone_col, "PLAYER", "FGT", "FG%"]]


def get_leading_perc_by_zone(df, zone_col="ZONE", threshold=0.75):
    """Calculate the leading FG% shooter by zone among the top volume shooters
    in the corresponing zone, determined by the `threshold` variable

    Args:
        df (pd.DataFrame): The dataframe of the shooting data
        zone_col (str): The column name of the zone data.
        threshold (float): A float between 0 and 1, representing the quantile of the
            top volume shooters

    Returns:
        pd.DataFrame: A dataframe with the leading player of FG%
            by zone
    """

    pvt_df = reshape_actions_player_by_zone(df, zone_col)

    thrslds_by_zone = pvt_df.groupby([zone_col])["FGT"].quantile(threshold)
    merged_df = pvt_df.merge(thrslds_by_zone.to_frame("threshold").reset_index())
    filtered_df = merged_df[merged_df["FGT"] >= merged_df["threshold"]]
    ordered_df = filtered_df.sort_values([zone_col, "FG%", "FGT"], ascending=[True, False, False])
    top_players = ordered_df.drop_duplicates(zone_col, keep="first")
    return top_players


In [13]:
season = 2024
competition = "E"
load_file = False

In [ ]:
if load_file:
    shot_df = pd.read_csv(f"../data/shooting_data_{competition}_{season}.csv")
else:
    shotdata = ShotData(competition)
    shot_df = shotdata.get_game_shot_data_single_season(season)
    shot_df.to_csv(f"../data/shooting_data_{competition}_{season}.csv", index=False)

# Find leading scorers by zone

In [23]:
latest_round = shot_df["Round"].max()
all_rounds = True
last_n = True  # Set to True if you want to filter the top N rounds
n = 5  # Number of top rounds to filter

In [24]:
shot_df = corner_three_zones(shot_df)
# shot_df = hexagon_zones(shot_df, gridsize=10)
# fg_perc_hex_heatmap(shot_df, gridsize=20)

In [25]:
zone_type = "ZONE_C"  # ZONE, ZONE_HEX

In [ ]:
if all_rounds:
   print("All rounds")
   top_players = get_leading_scorer_by_zone(shot_df, zone_col=zone_type)
   top_attempts = get_leading_attempts_by_zone(shot_df, zone_col=zone_type)
   top_percs = get_leading_perc_by_zone(shot_df, zone_col=zone_type, threshold=0.8)
   file_tag = f"up-to-round-{latest_round}"
elif last_n:
   print(f"Top {n} rounds")
   mask = (shot_df["Round"] > (latest_round - n))
   round_df =  shot_df[mask].reset_index(drop=True)
   top_players = get_leading_scorer_by_zone(round_df, zone_col=zone_type)
   top_attempts = get_leading_attempts_by_zone(round_df, zone_col=zone_type)
   top_percs = get_leading_perc_by_zone(round_df, zone_col=zone_type, threshold=0.8)
   file_tag = f"round-{latest_round}-last-{n}"
else:
   print(f"Round {latest_round}")
   mask = (shot_df["Round"] == latest_round)
   round_df =  shot_df[mask].reset_index(drop=True)
   top_players = get_leading_scorer_by_zone(round_df, zone_col=zone_type)
   top_attempts = get_leading_attempts_by_zone(round_df, zone_col=zone_type)
   top_percs = get_leading_perc_by_zone(round_df, zone_col=zone_type, threshold=0.8)
   file_tag = f"round-{latest_round}"

In [27]:
# top_percs

In [ ]:
plot_leading_scorers_by_zone(
    shot_df, top_percs, zone_col=zone_type,
    title="Top FG% shooters by zone",
    filename=f"../generated-plots/top-fg-perc-by-zone-{file_tag}-2024-2025.png"
)

In [ ]:
plot_leading_scorers_by_zone(
    shot_df, top_attempts, zone_col=zone_type,
    title="Top shooters by volume of FGs by zone",
    filename=f"../generated-plots/volume-fgs-by-zone-{file_tag}-2024-2025.png"
)

In [ ]:
plot_leading_scorers_by_zone(
    shot_df, top_players, zone_col=zone_type,
    title="Leading scorers by zone",
    filename=f"../generated-plots/leading-scorers-by-zone-{file_tag}-2024-2025.png"

)

# Player Shooting Chart

In [35]:
# player_name = 'NEDOVIC, NEMANJA'
# player_name = "LESSORT, MATHIAS"
# player_name = "SHORTS, TJ"
# player_name = "VEZENKOV, SASHA"
player_name = "NUNN, KENDRICK"

all_rounds = True
round_ = latest_round  # Set to None if you want all rounds of the season
last_n = True  # Set to True if you want to filter the last N rounds

In [ ]:
player_mask = (shot_df["PLAYER"] == player_name)
if all_rounds:
     player_mask &= (shot_df["Round"] == round_)
elif last_n:
     player_mask &= (shot_df["Round"] > (latest_round - n))
else:
     pass


player_shot_df = shot_df[player_mask]
palyer_missed_mask = player_shot_df["ID_ACTION"].str.contains("FGA")
player_made_mask = player_shot_df["ID_ACTION"].str.contains("FGM")

plot_scatter(player_shot_df[player_made_mask], player_shot_df[palyer_missed_mask], title=f'{player_name} - Shot Chart', color="red")

# Teams' shooting chart

In [13]:
from euroleague_api.team_stats import TeamStats

teamstats = TeamStats("E")
team_info = teamstats.get_team_stats_single_season(endpoint="traditional", season=2024, phase_type_code=None, statistic_mode="PerGame")
code_to_name = team_info[["team.code", "team.name"]].set_index("team.code").to_dict()["team.name"]

In [ ]:
code_to_name

In [15]:
team_abbr = "OLY"
round_ = None  # Set to None if you want all rounds of the season

In [ ]:
team_mask = (shot_df["TEAM"] == team_abbr)
if round_ is not None:
     team_mask &= (shot_df["Round"] == round_)

team_shot_df = shot_df[team_mask]
team_missed_mask = team_shot_df["ID_ACTION"].str.contains("FGA")
team_made_mask = team_shot_df["ID_ACTION"].str.contains("FGM")

plot_scatter(team_shot_df[team_made_mask], team_shot_df[team_missed_mask], title=f'{code_to_name[team_abbr]} - Shot Chart', color="red")

## Plot shooting charts of all teams

In [18]:
round_ = 19  # Set to None if you want all rounds of the season

In [ ]:
for team_abbr in shot_df["TEAM"].unique():
    team_mask = (shot_df["TEAM"] == team_abbr)
    if round_ is not None:
        team_mask &= (shot_df["Round"] == round_)
    team_shot_df = shot_df[team_mask]
    team_missed_mask = team_shot_df["ID_ACTION"].str.contains("FGA")
    team_made_mask = team_shot_df["ID_ACTION"].str.contains("FGM")
    plot_scatter(team_shot_df[team_made_mask], team_shot_df[team_missed_mask], title=f'{code_to_name[team_abbr]} - Shot Chart', color="red")